In [12]:
import requests
from io import StringIO
import pandas as pd

#爬月報表
def crawlMonthReport(date):
    
    #date.year-1911 轉民國
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(date.year-1911)+'_'+str(date.month)+'.html'
    r = requests.get(url)
    r.encoding = 'big5'
    dfs = pd.read_html(StringIO(r.text))
    
    #取得column name 
    #取df[6]只是舉例，因為它每個產業都有新的欄位名稱，[6]好像是塑膠產業
    collist = [] 
    for i in dfs[6].columns:
        collist.append(i[1])
        
    newdfs = []

    #在來開始找規律，和剛剛找欄為一樣，這個list的某些地方是存欄位或標題等名稱，因此只要看長度是否大於2就行
    for i in dfs:
        if len(i) > 2:
            newdfs.append(i)

    #把所有list合併成一DataFrame
    df = newdfs[0]
    for i in newdfs[1:]:
        df = pd.concat( [df, i], axis=0 )

    df.columns = collist
    
    #把合計刪掉
    df = df[~(df['公司代號'] == '合計')]
    
    #新增date欄位
    df['date'] = pd.to_datetime(datetime.date(date.year, date.month, 1))
    
    df = df.rename(columns={'公司代號':'stock_id'})
    df = df.set_index(['stock_id', 'date'])
    
    df = df.apply(lambda s:pd.to_numeric(s, errors='coerce'))
    
    #刪掉備註欄位
    df = df.drop(['公司名稱', '備註'], axis=1)
    
    return df


In [ ]:
import sqlite3
import os
import datetime

conn = sqlite3.connect('dataBase.db')

In [14]:
from dateutil.rrule import rrule, DAILY, MONTHLY

def month_range(start_date, end_date):
    return [dt.date() for dt in rrule(MONTHLY, dtstart=start_date, until=end_date)]

In [15]:
import time
def update_table(conn, table_name, crawl_function, dates):
    
    df = pd.DataFrame()
    dfs = {}
    
    for d in dates:
        
        print('正在爬取: ', d)
        
        data = crawl_function(d)
        
        if data is None:
            print('失敗，可能是假日')
        else:
            df = df.append(data)
            print(d, '成功')
        
        if data is not None:
            add_to_sql(conn, table_name, df)
            df = pd.DataFrame()
            print(d, '儲存成功')
        
        time.sleep(15)

In [16]:
def add_to_sql(conn, name, df):
    
    exist = table_exist(conn, name)
    ret = pd.read_sql('select * from ' + name, conn, index_col=['stock_id', 'date'])if exist else pd.DataFrame()
    ret = ret.append(df)
    ret.reset_index(inplace=True)
    ret['stock_id'] = ret['stock_id'].astype(str)
    ret['date'] = pd.to_datetime(ret['date'])
    ret = ret.drop_duplicates(['stock_id', 'date'], keep='last')
    ret = ret.sort_values(['stock_id', 'date']).set_index(['stock_id', 'date'])
    
    
    ret.to_csv('backup.csv')
    
    try:
        ret.to_sql(name, conn, if_exists='replace')
        
    except:
        ret = pd.read_csv('backup.csv', parse_dates=['date'], dtype={'stock_id':str})
        ret['stock_id'] = ret['stock_id'].astype(str)
        ret.set_index(['stock_id', 'date'], inplace=True)
        ret.to_sql(name, conn, if_exists='replace')
        print("失敗")
 

In [17]:
def table_exist(conn, table):
    return list(conn.execute("SELECT count(*) from sqlite_master where type='table' and name='"+table+"'"))[0][0] == 1

In [ ]:
fromdate = datetime.date(2017,1,1)
toDate = datetime.date(2017,12,1)
dates = month_range(fromdate, toDate)
update_table(conn, 'monthRevenue', crawlMonthReport, dates)



#更新紀錄2020/1~2020/6
#更新紀錄2019/1~2019/12
#更新紀錄2018/1~2018/12
#更新紀錄2017/1~2017/12